In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import json
import scipy 
import torch
from torch.utils.data import Dataset, DataLoader
import logging
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://cloud.

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-30 03:51:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-08-30 03:51:39 (11.2 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UA_War").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
#import data from aws s3 bucket for 1 day. This will be the starting dataframe to populate DB

from pyspark import SparkFiles
start_date = "0701"

url = f"https://databootcamps3bucket.s3.us-west-2.amazonaws.com/ua_war/UkraineWar/{start_date}_UATweets.csv.gz"
spark.sparkContext.addFile(url)

df_start_date = spark.read.option("delimiter", ",").option("encoding", "UTF-8").option("multiLine", True).option("escape", '"').csv(SparkFiles.get(f"{start_date}_UATweets.csv.gz"),  header=True, inferSchema=True)

# df.show(truncate=False) 
df_start_date_2 = df_start_date.na.fill("<empty>")

df_start_date_2.show(5)

+---+---------+---------------+--------------------+------------+---------+---------+-----------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+--------+-----------+--------------+----------+-----------------+---------------------+-----------------------+---------------------+-------------------+-----------------------+---------------+----------------+--------------------+----------------------+--------------------+
|_c0|   userid|       username|            acctdesc|    location|following|followers|totaltweets|       usercreatedts|            tweetid|      tweetcreatedts|retweetcount|                text|            hashtags|language|coordinates|favorite_count|is_retweet|original_tweet_id|original_tweet_userid|original_tweet_username|in_reply_to_status_id|in_reply_to_user_id|in_reply_to_screen_name|is_quote_status|quoted_status_id|quoted_status_userid|quoted_status_username|         extractedts|
+---+---------+-----

In [5]:
#see what the datatypes for df_8018 
df_start_date.dtypes

[('_c0', 'int'),
 ('userid', 'bigint'),
 ('username', 'string'),
 ('acctdesc', 'string'),
 ('location', 'string'),
 ('following', 'int'),
 ('followers', 'int'),
 ('totaltweets', 'int'),
 ('usercreatedts', 'string'),
 ('tweetid', 'bigint'),
 ('tweetcreatedts', 'string'),
 ('retweetcount', 'int'),
 ('text', 'string'),
 ('hashtags', 'string'),
 ('language', 'string'),
 ('coordinates', 'string'),
 ('favorite_count', 'int'),
 ('is_retweet', 'boolean'),
 ('original_tweet_id', 'bigint'),
 ('original_tweet_userid', 'bigint'),
 ('original_tweet_username', 'string'),
 ('in_reply_to_status_id', 'bigint'),
 ('in_reply_to_user_id', 'bigint'),
 ('in_reply_to_screen_name', 'string'),
 ('is_quote_status', 'boolean'),
 ('quoted_status_id', 'bigint'),
 ('quoted_status_userid', 'bigint'),
 ('quoted_status_username', 'string'),
 ('extractedts', 'string')]

In [6]:
#use this to determine which type of drop is needed. Also not all all dates have same number of columns 
len(df_start_date.columns)

29

In [7]:
from pyspark import SparkFiles
from pyspark.sql.functions import *

#use this to determine which type of drop is needed. Also not all all dates have same number of columns 
df_start_date_count = len(df_start_date.columns)
if df_start_date_count == 18:
  df_start_date_2 = df_start_date_2.withColumn("is_retweet", lit(None).cast('boolean')) 
  df_start_date_2 = df_start_date_2.withColumn("is_quote_status", lit(None).cast('boolean')) 

len(df_start_date_2.columns)

29

In [8]:
from pyspark.sql.functions import *

#1. Drop Uncessary Columns using the data set as a starting point. 
cleaned_df = df_start_date_2.drop("userid", "_c0", "acctdesc", "location", "tweetid", "coordinates", "original_tweet_id", "original_tweet_userid", "original_tweet_username", "in_reply_to_status_id", "in_reply_to_screen_name", 'in_reply_to_user_id', "quoted_status_id", "quoted_status_username", "quoted_status_userid")

#2. change columns from string to year:month:day date format
cleaned_df = cleaned_df.withColumn("usercreatedts",to_date("usercreatedts"))
cleaned_df = cleaned_df.withColumn("tweetcreatedts",to_date("tweetcreatedts"))
cleaned_df = cleaned_df.withColumn("extractedts",to_date("extractedts"))

#3. filter out langauge = english only
cleaned_df = cleaned_df.filter(cleaned_df["language"]=="en")
cleaned_df = cleaned_df.filter(cleaned_df["usercreatedts"] < "2009-01-01")

cleaned_df.show(5)

+---------------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
|       username|following|followers|totaltweets|usercreatedts|tweetcreatedts|retweetcount|                text|            hashtags|language|favorite_count|is_retweet|is_quote_status|extractedts|
+---------------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
|    Foxmental_X|     5623|     6231|     295444|   2008-08-24|    2022-07-01|           1|Cyberattack impac...|[{'text': 'News',...|      en|             1|     false|          false| 2022-07-01|
|      PSUdotcom|     1328|    46871|      82738|   2008-11-06|    2022-07-01|           1|You Can Now Sign ...|[{'text': 'Skate'...|      en|             1|     false|          false| 2022-07-01|
|digitalmaveric

In [9]:
cleaned_df_count = cleaned_df.count()
cleaned_df_count

2465

In [10]:
date_arr_july = [ "0702", "0703", "0704", "0705", "0706", "0707", "0708", "0709","0710", "0711", "0712", "0713", "0714", "0715", "0716", "0717", "0718", "0719", "0720", "0721", "0722", "0723", "0724", "0725", "0726", "0727", "0728", "0729", "0730", "0731"]

In [12]:
from pyspark import SparkFiles
from pyspark.sql.functions import *

url = "https://databootcamps3bucket.s3.us-west-2.amazonaws.com/ua_war/UkraineWar"

for day in date_arr_july:
      #load the data from aws
      aws_url = f"{url}/{day}_UATweets.csv.gz"
      spark.sparkContext.addFile(aws_url)
      temp_df = spark.read.option("delimiter", ",").option("encoding", "UTF-8").option("multiLine", True).option("escape", '"').csv(SparkFiles.get(f"{day}_UATweets.csv.gz"),  header=True, inferSchema=True)
      
      #keep count number of columns to determine which if else block it will hit
      temp_count = len(temp_df.columns)

      #change columns from string to year:month:day date format
      temp_df = temp_df.withColumn("usercreatedts",to_date("usercreatedts"))
      temp_df = temp_df.withColumn("tweetcreatedts",to_date("tweetcreatedts"))
      temp_df = temp_df.withColumn("extractedts",to_date("extractedts"))

      #filter out data for english only 
      temp_df = temp_df.filter(temp_df["language"]=="en")
      #filter out usercreated after 2009 
      temp_df = temp_df.filter(temp_df["usercreatedts"] < "2009-01-01")

      #fill in null values 
      temp_df_2 = temp_df.na.fill("<empty>")

      #some days the data columns has less columns then other days 
      if temp_count == 18:
        temp_df_3 = temp_df_2.drop("userid", "_c0", "acctdesc", "location", "tweetid", "coordinates")
        temp_df_3 = temp_df_3.withColumn("is_retweet", lit(None).cast('boolean')) 
        temp_df_3 = temp_df_3.withColumn("is_quote_status", lit(None).cast('boolean')) 

      elif temp_count == 29:
        temp_df_3 = temp_df_2.drop("userid", "_c0", "acctdesc", "location", "tweetid", "coordinates", "original_tweet_id", "original_tweet_userid", "original_tweet_username", "in_reply_to_status_id", "in_reply_to_screen_name", 'in_reply_to_user_id', "quoted_status_id", "quoted_status_username", "quoted_status_userid")

      else:
        print(f"Error on {day}_UATweets.csv.gz and column count {temp_count}")

      cleaned_df = cleaned_df.unionByName(temp_df_3)
      print(f"{day} - {cleaned_df.count()}")


0702 - 4472
0703 - 6429
0704 - 8948
0705 - 11385
0706 - 13883
0707 - 16407
0708 - 18710
0709 - 20634
0710 - 22707
0711 - 25609
0712 - 28065
0713 - 30304
0714 - 33377
0715 - 35998
0716 - 37844
0717 - 39926
0718 - 42128
0719 - 44276
0720 - 46643
0721 - 48280
0722 - 49857
0723 - 51448
0724 - 52760
0725 - 54117
0726 - 55491
0727 - 57097
0728 - 58889
0729 - 61154
0730 - 62848
0731 - 64518


In [13]:
cleaned_df.show(5)

+---------------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
|       username|following|followers|totaltweets|usercreatedts|tweetcreatedts|retweetcount|                text|            hashtags|language|favorite_count|is_retweet|is_quote_status|extractedts|
+---------------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+
|    Foxmental_X|     5623|     6231|     295444|   2008-08-24|    2022-07-01|           1|Cyberattack impac...|[{'text': 'News',...|      en|             1|     false|          false| 2022-07-01|
|      PSUdotcom|     1328|    46871|      82738|   2008-11-06|    2022-07-01|           1|You Can Now Sign ...|[{'text': 'Skate'...|      en|             1|     false|          false| 2022-07-01|
|digitalmaveric

In [14]:
#add week column for "tweetcreatedts"
cleaned_df = cleaned_df.withColumn("weekofyear",weekofyear("tweetcreatedts"))

In [15]:
cleaned_df.show(5)

+---------------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+----------+
|       username|following|followers|totaltweets|usercreatedts|tweetcreatedts|retweetcount|                text|            hashtags|language|favorite_count|is_retweet|is_quote_status|extractedts|weekofyear|
+---------------+---------+---------+-----------+-------------+--------------+------------+--------------------+--------------------+--------+--------------+----------+---------------+-----------+----------+
|    Foxmental_X|     5623|     6231|     295444|   2008-08-24|    2022-07-01|           1|Cyberattack impac...|[{'text': 'News',...|      en|             1|     false|          false| 2022-07-01|        26|
|      PSUdotcom|     1328|    46871|      82738|   2008-11-06|    2022-07-01|           1|You Can Now Sign ...|[{'text': 'Skate'...|      en|             1|     false|

In [16]:
cleaned_df.dtypes

[('username', 'string'),
 ('following', 'int'),
 ('followers', 'int'),
 ('totaltweets', 'int'),
 ('usercreatedts', 'date'),
 ('tweetcreatedts', 'date'),
 ('retweetcount', 'int'),
 ('text', 'string'),
 ('hashtags', 'string'),
 ('language', 'string'),
 ('favorite_count', 'int'),
 ('is_retweet', 'boolean'),
 ('is_quote_status', 'boolean'),
 ('extractedts', 'date'),
 ('weekofyear', 'int')]

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [17]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com:5432/ua_data"
config = {"user":"uatweets", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [18]:
# Write review_id_df to table in RDS
cleaned_df.write.jdbc(url=jdbc_url, table='tweets_table', mode=mode, properties=config)